In [84]:
import pandas as pd 
import numpy as np
import nltk
# 1: +ve, 0: -ve

In [85]:
import json

with open('dataset/strong-negatives.json') as fopen:
    myfile1 = json.load(fopen)

with open('dataset/strong-positives.json') as fopen:
    myfile2 = json.load(fopen)

In [86]:
negative_data = pd.DataFrame(myfile1, columns = ['text'])
# negative_data = negative_data[:100000]
negative_data['label'] = 0
print(negative_data.head(10))
print(len(negative_data))

                                                text  label
0  @andcrra Ngelamar kasih cincin tp kok mukanya ...      0
1  @mojokdotco Caption iki nggarai uwong males ni...      0
2  Valentino Rossi Tidak Setuju Kompetisi MotoGP ...      0
3  Loop in nama dlm email pon boleh jd issue... D...      0
4                             Hilang nyawaku aku tgk      0
5             @AyekKamal yer lah sbb sombong mmg lah      0
6  Guys, tolong rt tweet ni sampai owner dia dapa...      0
7  Aku ada motor racing ,\naku bawa ronda ,\nawek...      0
8  - STILL 17 -\nSEDIHBGT!!!! Kebayang kan betapa...      0
9  @jokowi Bangga manfaat "DILAN" perputaran uang...      0
1892193


In [87]:
positive_data = pd.DataFrame(myfile2, columns = ['text'])
# positive_data = positive_data[:100000]
positive_data['label'] = 1
print(positive_data.head(10))
print(len(positive_data))

                                                text  label
0  @nasikebuli07 @FANBASEBOKEP2 Dom jakpus sih, b...      1
1                  @seolasidooo Jujur kacang ijo !!       1
2            Sahur tengah malam kaya nya enak ya...       1
3  @Ini_Talkshow @tiket\n\nMakan serabi enak pas ...      1
4  @imtaeyonglee loh kenapa? kan marga oppa juga ...      1
5  Resort Datuk Jhon Gani..kuala penyu..boleh baw...      1
6  |190506| BTS at Rose Bowl Day 1|\n\n Jadi mula...      1
7  I'm at Menara Axis in Petaling Jaya, Selangor ...      1
8  Usually aku rimas tngok parking kiri kanan\n\n...      1
9  Perlu hijrah sebentar dari sini untuk menenang...      1
1084592


In [88]:
news_sentiment = pd.read_csv('./dataset/sentiment-data-v2.csv')
news_sentiment.loc[news_sentiment['label'] == 'Positive', 'label'] = 1
news_sentiment.loc[news_sentiment['label'] == 'Negative', 'label'] = 0
news_sentiment['label'] = news_sentiment['label'].astype('int64')
news_sentiment = news_sentiment[['text', 'label']]
print(news_sentiment.head(10))
print(len(news_sentiment))

                                                text  label
0  Lebih-lebih lagi dengan  kemudahan internet da...      0
1  boleh memberi teguran kepada parti tetapi perl...      1
2  Adalah membingungkan mengapa masyarakat Cina b...      0
3  Kami menurunkan defisit daripada 6.7 peratus p...      1
4        Ini masalahnya. Bukan rakyat, tetapi sistem      0
5  Masyarakat Cina dapat melihat bagaimana peranc...      1
6  Tetapi penyelenggaraan ini ada skop tertentu y...      1
7  Jika memilih untuk tidak dikritik, duduk saja ...      1
8  Ini membawa erti haram juga untuk sokong BN ke...      0
9  Jika anda tidak percaya, tanya ibu bapa di Pul...      1
3685


In [89]:
# Combine both dataframes into one master dataframe
data = pd.concat([negative_data, positive_data, news_sentiment], ignore_index = True)
print(data.head(10))
print(len(data))

                                                text  label
0  @andcrra Ngelamar kasih cincin tp kok mukanya ...      0
1  @mojokdotco Caption iki nggarai uwong males ni...      0
2  Valentino Rossi Tidak Setuju Kompetisi MotoGP ...      0
3  Loop in nama dlm email pon boleh jd issue... D...      0
4                             Hilang nyawaku aku tgk      0
5             @AyekKamal yer lah sbb sombong mmg lah      0
6  Guys, tolong rt tweet ni sampai owner dia dapa...      0
7  Aku ada motor racing ,\naku bawa ronda ,\nawek...      0
8  - STILL 17 -\nSEDIHBGT!!!! Kebayang kan betapa...      0
9  @jokowi Bangga manfaat "DILAN" perputaran uang...      0
2980470


In [90]:
# save data in csv file
# data.to_csv('bahasa_sentiment.csv', encoding='utf-8',index=False)

In [91]:
data['label'].value_counts()

0    1893488
1    1086982
Name: label, dtype: int64

In [92]:
# handle missing values
data = data.dropna(axis = 0, how ='any')

In [93]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import Counter
import string
import re
import stopwordsiso as sw

# download required library from nltk
# nltk.download('punkt')

# create stemmer for bahasa
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stop words consist of malay, indo, english
stop_words_main = list(sw.stopwords(["ms", "id", "en"]))
# custom stopwords such as shortform
stop_words_custom = ['kau', 'yg', 'mcm', 'gak', 'nak', 'ni', 'tu', 'la', 'je', 'kat', 'ya', 'dgn', 'tau', 'org', 'rt', 'aja', 'nk', 'dah',
                        'orang', 'sy', 'ga', 'kalo', 'kena']
STOP_WORDS = np.unique(stop_words_main+stop_words_custom)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def text_preprocessing(text):

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # remove links
    text = re.sub('http[s]?://\S+', '', text)

    # remove word with tweethandle @name
    text = re.sub('[^ ]*@[^ ]*', '', text)

    # remove emoji
    text = remove_emoji(text)

    # tokennization
    tokens = word_tokenize(text)

    # stemmer and remove punctuation
    words = []
    for token in tokens:
        if token not in string.punctuation:
            temp = stemmer.stem(token)
            words.append(temp)

    # remove stopwords
    cleaned = []
    for word in words:
        if word not in STOP_WORDS:
            cleaned.append(word)

    # join all words into a complete sentence 
    complete_sentence = ' '.join([str(word) for word in cleaned])

    # remove extra line spaces between words in a sentence
    complete_sentence = " ".join(complete_sentence.split())
    
    return complete_sentence

In [94]:
sample = '@natasya Aku pernah first day raya tudung aku terbakar 😩😩😩.... nasib baik tak moody sebab kecik je bekas bakar. https://t.co/24DGRbL70Z'
text_preprocessing(sample)

'day tudung bakar nasib moody kecik bakar'

In [95]:
text_cleaning = lambda x: text_preprocessing(x)
data['cleaned_Text'] = pd.DataFrame(data['text'].apply(text_cleaning))
data['cleaned_Text'].head(10)

0    ngelamar kasih cincin muka songong sedih gue liat
1    iki nggarai uwong males nikah min manusia arep...
2          valentino rossi tuju kompetisi motogp eropa
3    loop nama dlm email pon jd issue email reply j...
4                                     hilang nyawa tgk
5                                  yer sbb sombong mmg
6    guys tolong tweet owner phone tinggal teks pak...
7    motor racing bawa ronda awek bonceng dar dar l...
8       sedihbgt bayang betapa sedih gimana thn seteru
9    bangga manfaat dil putar uang dukung tumbuh mi...
Name: cleaned_Text, dtype: object

In [96]:
# SPLIT TRAINING & TESTING DATA
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['cleaned_Text'],data['label'],test_size=0.2,shuffle=True, random_state=42)
print(X_train.shape, y_train.shape)

(2384376,) (2384376,)


In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, f1_score, recall_score

def sentiment_pipeline(data_train_input,data_train_target,model_type):
    # Classifier selection
    if model_type == "linear":
        classifier = LinearSVC()
    elif model_type == "logistic":
        classifier = LogisticRegression(max_iter=1000)
    elif model_type == "sgd":
        classifier = SGDClassifier()
    elif model_type == "naive_bayes":
        classifier = MultinomialNB()
    elif model_type == "xgboost":
        classifier = XGBClassifier(use_label_encoder=False,eta=0.1,gamma=0.3, n_estimators=100, learning_rate=0.5, min_child_weight=5, 
        max_depth=5, colsample_bytree=0.7,objective="binary:logistic", eval_metric="logloss",verbosity=0)

    tfidf = TfidfVectorizer()

    # Pipeline setup
    clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

    model = clf.fit(data_train_input,data_train_target)

    return model

def sentiment_model_predict(model,data_test_input,data_test_target):
    data_prediction=model.predict(data_test_input)
    conf_matrix = confusion_matrix(data_test_target,data_prediction)
    acc_score = accuracy_score(data_test_target, data_prediction)
    pre_score = precision_score(data_test_target, data_prediction, average="macro")
    re_score = recall_score(data_test_target, data_prediction, average="macro")
    f_score = f1_score(data_test_target, data_prediction, average="macro")

    print("Accuracy : "+str(round(acc_score*100,2)))
    print("Precision : "+str(round(pre_score*100,2)))
    print("Recall : "+str(round(re_score*100,2)))
    print("F1-Score :"+str(round(f_score*100,2)))
    print(conf_matrix)

In [98]:
# Support Vector Classification
svm_model = sentiment_pipeline(X_train, y_train, 'linear')
sentiment_model_predict(svm_model,X_test,y_test)

Accuracy : 94.7
Precision : 94.04
Recall : 94.66
F1-Score :94.33
[[358735  19618]
 [ 11949 205792]]


In [99]:
# Logistic Regression
lr_model = sentiment_pipeline(X_train, y_train, 'logistic')
sentiment_model_predict(lr_model,X_test,y_test)

Accuracy : 94.75
Precision : 94.11
Recall : 94.67
F1-Score :94.38
[[359334  19019]
 [ 12269 205472]]


In [100]:
# Stochastic Gradient Descent
sgd_model = sentiment_pipeline(X_train, y_train, 'sgd')
sentiment_model_predict(sgd_model,X_test,y_test)

Accuracy : 90.64
Precision : 90.9
Recall : 88.76
F1-Score :89.65
[[362137  16216]
 [ 39603 178138]]


In [101]:
# Multinomial Naive Bayes
nb_model = sentiment_pipeline(X_train, y_train, 'naive_bayes')
sentiment_model_predict(nb_model,X_test,y_test)

Accuracy : 87.96
Precision : 89.67
Recall : 84.56
F1-Score :86.23
[[367618  10735]
 [ 61064 156677]]


In [102]:
# Xgboost
xg_model = sentiment_pipeline(X_train, y_train, 'xgboost')
sentiment_model_predict(xg_model,X_test,y_test)

Accuracy : 84.03
Precision : 86.43
Recall : 79.42
F1-Score :81.25
[[365284  13069]
 [ 82111 135630]]


In [103]:
import joblib

# save model
joblib_file_svm = "model/bahasa_sentiment_svm_model2.pkl"
joblib.dump(svm_model, joblib_file_svm)

['model/bahasa_sentiment_svm_model2.pkl']

In [104]:
# load model
joblib_SVM_model = joblib.load(joblib_file_svm)
sentiment_model_predict(joblib_SVM_model,X_test,y_test)

Accuracy : 94.7
Precision : 94.04
Recall : 94.66
F1-Score :94.33
[[358735  19618]
 [ 11949 205792]]


In [105]:
# joblib_file_lr = "model/bahasa_sentiment_lr_model2.pkl"
# joblib.dump(lr_model, joblib_file_lr)

In [106]:
# joblib_LR_model = joblib.load(joblib_file_lr)
# sentiment_model_predict(joblib_LR_model,X_test,y_test)